In [1]:
import requests
import os
from glob import glob
from pprint import pprint

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext
)
from llama_index.core import SummaryIndex, Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.objects import ObjectIndex

In [5]:
pty_docs_url = "https://docs.python.org/3/library/pty.html"
pty_blog_example_url = "https://allican.be/blog/2017/01/15/python-dummy-serial-port.html"
pty_docs_dirpath = "../../manuals/pty_docs"

In [6]:
# read info from pty docs website
pty_docs_filepath = os.path.join(pty_docs_dirpath, "pty_docs.html")
pty_docs_response = requests.get(pty_docs_url)
with open(pty_docs_filepath, "w") as f:
    f.write(pty_docs_response.text)

pty_blog_filepath = os.path.join(pty_docs_dirpath, "pty_blog.html")
pty_blog_response = requests.get(pty_blog_example_url)
with open(pty_blog_filepath, "w") as f:
    f.write(pty_blog_response.text)


In [7]:
pty_docpaths = {
    "pty_docs": pty_docs_filepath,
    "pty_blog": pty_blog_filepath,
}

In [10]:
# set up parser
parser = LlamaParse(
    result_type="markdown"  # "markdown" and "text" are available
)

# use SimpleDirectoryReader to parse manuals
pty_parsed_docs = {}
for doc_name, doc_path in pty_docpaths.items():
    file_extractor =  {".html": parser}
    pty_parsed_docs[doc_name] = SimpleDirectoryReader(input_files=[doc_path], file_extractor=file_extractor).load_data()


Started parsing the file under job_id db0500e2-a6fb-4c82-8363-ffa4e04ef055
Started parsing the file under job_id 5af24162-78e6-4d4c-9d64-b5b6518f0331


In [8]:
Settings.llm = OpenAI(temperature=0, model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [11]:
node_parser = SentenceSplitter()

persist_dir = "./storage/pty"

if not os.path.exists(persist_dir):
    os.makedirs(persist_dir)

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

# Build agents dictionary
for doc_name, doc in pty_parsed_docs.items():

    print(f"Building agent for document {doc_name}")
    
    nodes = node_parser.get_nodes_from_documents(doc)
    all_nodes.extend(nodes)

    doc_persist_dir = os.path.join(persist_dir, doc_name)
    if not os.path.exists(doc_persist_dir):
        # build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(
            persist_dir=doc_persist_dir
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=doc_persist_dir),
        )

    # build summary index
    summary_index = SummaryIndex(nodes)
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
    summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)
    
    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=f"This tool contains specific information related to {vector_query_engine.query('Finish the last sentence in this message, but don''t include the prompt. Be as specific as possible but not verbose. This tool contains information related to ').response}",
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=f"This tool contains summary information about {vector_query_engine.query('Finish the last sentence in this message, but don''t include the prompt. Be as specific as possible but not verbose. This tool contains information related to ').response}",
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-4o")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to aid in tasks like the following {vector_query_engine.query('Finish the last sentence in this message, but don''t include the prompt. Be as specific as possible but not verbose. This document is useful for accomplishing tasks like the following ').response} \
You must ALWAYS use at least one of the tools provided when answering questions to help users accomplish these tasks; do NOT rely on prior knowledge.\
""",
    )

    agents[doc_name] = agent
    query_engines[doc_name] = vector_index.as_query_engine(
        similarity_top_k=5
    )

Building agent for document pty_docs
Building agent for document pty_blog


In [12]:
# define tool for each document agent
all_tools = []
for doc_name, _ in pty_parsed_docs.items():
    doc_summary = f"This tool is to aid in tasks like the following: {vector_query_engine.query('Finish the last sentence in this message, but don''t include the prompt. Be as specific as possible but not verbose. This document is useful for accomplishing tasks like the following ').response}"
    doc_tool = QueryEngineTool(
        query_engine=agents[doc_name],
        metadata=ToolMetadata(
            name=f"tool_{doc_name}",
            description=doc_summary,
        ),
    )
    all_tools.append(doc_tool)

In [13]:
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [14]:
pty_simulation_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=f""" \
You are an agent that will help a user write code to simulate serial communcation. \
You have tools that will help you write this code. These tools provide documentation and examples of using the python library pty to do this. \
Please always use the tools provided to answer a question. Do not rely on prior knowledge. \

""",
    verbose=True,
)

In [15]:
response = pty_simulation_agent.query("Write a python class that simulates a serial device. It should capture reads, and it should have the ability to write if necessary. This will be base class for simulating serial communication with specific hardware.")

Added user message to memory: Write a python class that simulates a serial device. It should capture reads, and it should have the ability to write if necessary. This will be base class for simulating serial communication with specific hardware.
=== Calling Function ===
Calling function: tool_pty_blog with args: {"input": "python class that simulates a serial device"}
Added user message to memory: python class that simulates a serial device
=== Calling Function ===
Calling function: vector_tool with args: {"input": "Python class that simulates a serial device"}
Got output: To simulate a serial device in Python, you can use the `pty` module to create pseudoterminals. Here is an example of how you can set up a dummy serial port that listens for commands and responds accordingly:

```python
import os, pty
from serial import Serial
import threading

class DummySerialDevice:
    def __init__(self):
        self.master, self.slave = pty.openpty()
        self.slave_name = os.ttyname(self.sla

In [16]:
pprint(response.response)

('Based on the information gathered, here is a Python class that simulates a '
 'serial device. This class captures reads and has the ability to write if '
 'necessary. It uses the `pty` module to create a pseudo-terminal pair and '
 'includes methods for writing to and reading from the simulated device.\n'
 '\n'
 '```python\n'
 'import os\n'
 'import pty\n'
 'import threading\n'
 '\n'
 'class SerialDeviceSimulator:\n'
 '    def __init__(self):\n'
 '        # Open a new pseudo-terminal pair\n'
 '        self.master_fd, self.slave_fd = pty.openpty()\n'
 '        self.slave_name = os.ttyname(self.slave_fd)\n'
 '        self.listener_thread = threading.Thread(target=self.listener)\n'
 '        self.listener_thread.start()\n'
 '\n'
 '    def listener(self):\n'
 '        while True:\n'
 '            res = b""\n'
 '            while not res.endswith(b"\\r\\n"):\n'
 '                res += os.read(self.master_fd, 1)\n'
 '            print("command: %s" % res)\n'
 "            if res == b'QPGS

In [17]:
response = pty_simulation_agent.query("That's a great start. Would it work to connect to this device using `pyserial`? So if other code wants to use the hardware, we can swap this in instead and the code on the user end remains the same, that is, using `pyserial` to connect given the port?")

Added user message to memory: That's a great start. Would it work to connect to this device using `pyserial`? So if other code wants to use the hardware, we can swap this in instead and the code on the user end remains the same, that is, using `pyserial` to connect given the port?
=== Calling Function ===
Calling function: tool_pty_docs with args: {"input": "simulate serial communication using pty and pyserial"}
Added user message to memory: simulate serial communication using pty and pyserial
=== Calling Function ===
Calling function: vector_tool with args: {"input": "simulate serial communication using pty and pyserial"}
Got output: To simulate serial communication using `pty` and `pyserial`, you can create a pair of pseudo-terminals using the `pty` module and then use `pyserial` to open and communicate through these terminals. Here is an example of how you can achieve this:

1. **Create a pair of pseudo-terminals:**

```python
import os
import pty
import serial

# Create a pair of p

In [18]:
pprint(response.response)

('Yes, you can simulate serial communication using `pty` and `pyserial` in '
 'Python. This allows you to create a dummy serial port that can be used for '
 'testing purposes. Here are two approaches to achieve this:\n'
 '\n'
 '### Approach 1: Basic Example\n'
 '\n'
 '1. **Create a pair of pseudo-terminals:**\n'
 '\n'
 '```python\n'
 'import os\n'
 'import pty\n'
 '\n'
 '# Create a pair of pseudo-terminals\n'
 'master, slave = pty.openpty()\n'
 'slave_name = os.ttyname(slave)\n'
 '```\n'
 '\n'
 '2. **Open the slave end using `pyserial`:**\n'
 '\n'
 '```python\n'
 'import serial\n'
 '\n'
 '# Open the slave end using pyserial\n'
 'ser = serial.Serial(slave_name, baudrate=9600, timeout=1)\n'
 '```\n'
 '\n'
 '3. **Write and read data:**\n'
 '\n'
 '```python\n'
 '# Write data to the serial port\n'
 "ser.write(b'Hello, World!\\n')\n"
 '\n'
 '# Read data from the serial port\n'
 'data = os.read(master, 100)\n'
 "print('Received:', data.decode())\n"
 '```\n'
 '\n'
 '4. **Close the serial port:

In [19]:
for tool in all_tools:
    pprint(tool.metadata)

ToolMetadata(description='This tool isto aid in tasks like the following '
                         'creating dummy serial ports in Python for testing '
                         'and reverse-engineering serial communication with '
                         'devices like solar inverters.',
             name='tool_pty_docs',
             fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>,
             return_direct=False)
ToolMetadata(description='This tool isto aid in tasks like the following '
                         'creating dummy serial ports in Python using '
                         'pseudoterminals for testing and reverse-engineering '
                         'serial communication.',
             name='tool_pty_blog',
             fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>,
             return_direct=False)
